In [3]:
__author__ = 'David Nidever, NOAO Data Lab Team'
__version__ = '20170602' # yyyymmdd

# Data Lab Interface Introduction
*David Nidever & the NOAO Data Lab Team*

<a class="anchor" id="toc"></a>
# Table of Contents
* [Getting started, module imports and setup](#imports)
* [Help](#help)
* [Authentication](#authentication)
* [Queries](#queries)
 * [Format](#queries.format)
 * [Synchronous versus Asynchronous](#queries.sync)
 * [Query History](#queries.history)
 * [Outputs](#queries.outputs)
* [Database Commands](#dbcommands)
 * [Database Schema](#dbcommands.schema)
 * [List MyDB Tables](#dbcommands.list)
 * [Drop a MyDB Table](#dbcommands.droptable)
 * [Export a MyDB Table to File](#dbcommands.exporttable)
 * [Simple Image Access (SIA) Search](#dbcommands.siaquery)
* [Storage](#storage)
 * [File Listing](#storage.ls)
 * [Unix-like File Commands - cp, mv, rm, mkdir, rmdir, ln](#storage.unixcmds)
 * [Copying files to/from VOSpace](#storage.sync)
 * [Copying File from URL](#storage.copyurl)
 * [Loading files into python](#storage.load)
 * [Saving data from python to files](#storage.save)

The Data Lab (DL) Interface is a user-friendly way of interacting with the various Data Lab services including authentication, database queries, and interacting with the virtual storage (VOSpace) your personal database (mydb).  This document is an introduction on how to use the Data Lab Interface.  **Note that to take full advantage of all the DL Interface funcionality it is recommended that these commands be run in a local python instance of Jupyter notebook on your laptop or desktop.**

<a class="anchor" id="imports"></a>
# Getting started, module imports and setup - the secret sauce
First, you need a copy of the "datalab" package.  This is already installed on the Data Lab notebook server.
To get started using the DL Interface, Import the DL interface class from the datalab package and create the "dl" object which is what you will use for the Data Lab commands.

In [1]:
from dl.dlinterface import Dlinterface
dl = Dlinterface()

Welcome to the Data Lab python interface.  Type dl.help() for help.


<a class="anchor" id="help"></a>
# Most importantly, Help!
The most important thing to know about a new package is how to get help!  There as several ways of doing this in the DL Interface.  If you simply type `dl.help()` it prints out the available commands. Note that every DL Interface command requires `()` at the end since they are object "methods".

In [2]:
dl.help()

The Data Lab python interface.
 
The available commands are:
 
dl.help()      - Helpful information
Use dl.help(<command>) for specific help on a command.
 
-- Login and authentication --
dl.login()          - Login to the Data Lab
dl.logout()         - Logout of the Data Lab
dl.status()         - Report on the user status
dl.whoami()         - Print the current active user
dl.servicestatus()  - Report on the status of the DL services
 
-- File system operations --
dl.ls()        - List a location in Data Lab
dl.get()       - Get a file from Data Lab
dl.put()       - Put a file into Data Lab
dl.cp()        - Copy a file in Data Lab
dl.mv()        - Move a file in Data Lab
dl.rm()        - Delete a file in Data Lab
dl.mkdir()     - Create a directory in Data Lab
dl.rmdir()     - Delete a directory in Data Lab
dl.ln()        - Link a file in Data Lab
dl.tag()       - Tag a file in Data Lab
 
-- Query and database operations --
dl.query()          - Query a remote data service in the Data

You can also get help on any specific command by doing `dl.help('COMMAND')`.  For example,

In [3]:
dl.help('cp')

Help on method cp in module dl.dlinterface:

cp(self, source=None, destination=None, verbose=True) method of dl.dlinterface.Dlinterface instance
    Copy a file in Data Lab VOSpace.
    
    Parameters
    ----------
    source : str
         The name of the file in VOSpace to copy, e.g. ``file1.txt``.
    
    destination : str
         The new name of the file in VOSpace, e.g. ``newfile1.txt``.
    
    Example
    -------
    
    Copy the file ``file.txt`` to ``newfile.txt``.
    
    .. code-block:: python
    
        dl.ls()
        file1.txt
    
        dl.cp('file1.txt','newfile.txt')
    
        dl.ls()
        file1.txt, newfile.txt



You can also use the standard python `help(function, object or object method)` syntax, e.g. `help('dl.logout')`, to get help, or putting a `?` at the end of a command.

In [4]:
dl.query?

Finally, since "dl" is an object you type `dl.<TAB>` and the available object methods will appear.

Finally, the DL Interface commands that require some inputs will return a simple syntax/signature statement if called with no arguments.

In [5]:
dl.query()

Syntax - dl.query(query, qtype='sql|adql', fmt='csv|string|array|structarray|pandas|table|votable|fits|hdf5',
                  out='', async=False, profile='default')


<a class="anchor" id="authentication"></a>
# Authentication
To get most of the functionality of the Data Lab you will need to login to your Data Lab account.  We do this with `dl.login()`.  You will be prompted for your password (and your username if you didn't specify it).

In [6]:
dl.login()

Enter user: username
Enter password: ········
Welcome to the Data Lab, username


When you are done with your Data Lab work you can logout with `dl.logout()`.  You can also see what your login status is with `dl.status()` and who you are currently logged-in as with `dl.whoami()`.

<a class="anchor" id="queries"></a>
# Database Queries
Let's get started with some database queries.  To see what catalogs are in the Data Lab database use `dl.schema()`.  You can also get information at deeper levels by specifying specific tables or columns, e.g. `dl.schema('smash_dr1.object')`.

In [7]:
dl.schema()


     Schema Name   Description
     -----------   -----------
        gaia_dr1   GAIA Data Release 1
            ivoa   IVOA ObsCore tables
        des_sva1   DES SVA1 Data Products
      tap_schema   TAP Schema Tables
            usno   USNO Astrometry Catalogs
       sdss_dr13   
         neo_dr1   NEO Survey Data Release 1
          ls_dr3   The DECam Legacy Survey Data Release 3
       smash_dr1   SMASH Data Release 1



The only requirement for a query is the SQL query string that specifies the columns you want, from which table and any restrictions or cuts.  `select COLUMNS from CATALOG.TABLE where CONSTRAINTS`.  It's often helpful for testing to end with a `LIMIT` statement to only return a small number of rows.

In [8]:
res = dl.query('select id,ra,dec,gmag from smash_dr1.object limit 5')
print res

Returning CSV formatted table as a string
id,ra,dec,gmag
84.43705,157.576594556189,-31.1810517321523,25.0617
84.43705,157.576594556189,-31.1810517321523,25.0617
84.41595,157.577066096003,-31.1810160528312,25.1985
84.41595,157.577066096003,-31.1810160528312,25.1985
84.39718,157.567844754001,-31.1777341604321,24.9557



<a class="anchor" id="queries.format"></a>
### Formats
There are several output formats that can be specified with the `fmt=` keyword.  The current available ones are:
* 'csv'  -   the returned result is a comma-separated string that looks like a csv file (newlines at the end of every row)
* 'ascii' -  same as csv but tab-delimited
* 'array' -  Numpy array
* 'structarray' - Numpy structured / record array
* 'pandas' - a Pandas data frame
* 'table' -  in Astropy Table format
* 'votable' -  result is a string XML-formatted as a VO table
* 'fits'  -  fits format, for file output ONLY
* 'hdf5'  -  HDF5 format, for file output ONLY

In [9]:
res = dl.query('select id,ra,dec,gmag from smash_dr1.object limit 5',fmt='structarray')
print res

Returning Numpy structured / record array
[(84.43705, 157.576594556189, -31.1810517321523, 25.0617)
 (84.43705, 157.576594556189, -31.1810517321523, 25.0617)
 (84.41595, 157.577066096003, -31.1810160528312, 25.1985)
 (84.41595, 157.577066096003, -31.1810160528312, 25.1985)
 (84.39718, 157.567844754001, -31.1777341604321, 24.9557)]


<a class="anchor" id="queries.sync"></a>
### Synchronous versus Asynchronous queries
There are two types of Data Lab queries that you can run: "synchronous" or "asynchronous".  A synchronous query will return directly and is akin to a "foreground" job, while an asynchronous query runs in the "background" and the results need to retrieved when the query is complete.  Queries are synchronous by default but asynchronous can be used by specifying `async=True`.  It is a good idea to use asynchronous if the query is expected to be quite long.

Let's try a simple asynchronous query.

In [10]:
jobid = dl.query('select id,ra,dec,gmag from smash_dr1.object limit 1000',fmt='structarray',async=True)

Asynchronous query JobID = ux2dp0tkw3o1x3e6 


When you run an asynchronous query it does not return the results but instead a JobID string.  This can later be used to see the status of the query with `dl.querystatus(JobID)` and/or retrive the results with `dl.queryresults(JobID)`.  You can also retrive async query results with the Query ID (QID) that can be obtained from the query history output (see below).

In [11]:
# Check the status of the query
dl.querystatus(jobid)

COMPLETED


In [12]:
# When it is completed retrieve the results
res = dl.queryresults(jobid)

Returning Numpy structured / record array


<a class="anchor" id="queries.history"></a>
### Query History
The DL Interface keeps a history of all the queries that you make, synchronous or asynchronous.  This can be useful for seeing what you have done, checking the status of any async queries, or rerunning queries.

In [13]:
dl.queryhistory()

-------------------------------------------------------------------------------------------------------------------
QID          DATE        Type  A/SYNC  Format       Status          JobID                   Query
-------------------------------------------------------------------------------------------------------------------
1    2017-05-25 18:30:08  sql   ASYNC  structarray  COMPLETED   ux2dp0tkw3o1x3e6    'select id,ra,dec,gmag from smash_dr1.object limit 1000'
-------------------------------------------------------------------------------------------------------------------


The QID (first column) can be used to retrive async query results, e.g. `dl.queryresults(1)`, or to rerun a previous query `res = dl.query(1,fmt='pandas')`.

<a class="anchor" id="queries.outputs"></a>
## Outputs
There are various ways to output the results of a query using the `out=` keyword.
* Direct output to the user at the prompt.  Don't specify `out` or set to `None`.
* To a local file. `out=FILENAME`.
* To a file on VOSpace.  `out=vos://FILENAME`.
* To a table in your mydb.  `out=mydb://TABLENAME`.

In [14]:
dl.query('select * from smash_dr1.object limit 5000',out='vos://smash_test.csv')

In [15]:
# Check that the new file is there in your vospace
dl.ls(verbose=True)

drw-rw----  dnidever    512B  02 Feb 2017 00:00:00  public/  
-rw-rw----  dnidever  152.9K  02 Jun 2017 13:03:07  results.hdf5  
-rw-rw-r--  dnidever    1.2M  02 Jun 2017 15:16:50  smash_test.csv  
-rw-rw----  dnidever  234.4K  01 Jun 2017 12:20:52  test.csv  
-rw-rw----  dnidever  317.8K  01 Jun 2017 12:21:13  test.fits  
-rw-rw----  dnidever  234.4K  01 Jun 2017 12:20:52  test.txt  
drw-rw----  dnidever    512B  02 Feb 2017 00:00:00  tmp/  


<a class="anchor" id="dbcommands"></a>
# Other Database Commands
There are several other useful database commands.

<a class="anchor" id="dbcommands.schema"></a>
### Database Schema
As already mentioned above, the `dl.schema()` command can be used to learn more about the various database catalogs and tables.  It's possible to obtain information at various levels.  The input is `catalog.table.column`.

In [16]:
# List top level catalogs
dl.schema()


     Schema Name   Description
     -----------   -----------
        gaia_dr1   GAIA Data Release 1
            ivoa   IVOA ObsCore tables
        des_sva1   DES SVA1 Data Products
      tap_schema   TAP Schema Tables
            usno   USNO Astrometry Catalogs
       sdss_dr13   
         neo_dr1   NEO Survey Data Release 1
          ls_dr3   The DECam Legacy Survey Data Release 3
       smash_dr1   SMASH Data Release 1



In [17]:
# List tables in smash_dr1 catalog
dl.schema('smash_dr1')


Schema: smash_dr1

      Table Name   Description
      ----------   -----------
            chip   Info on each chip in the frame
        exposure   Info on each exposure
           field   Info on each target field (position, Num exposures, etc)
          object   Average photometry of each unique object
          source   All of the individual source measurements
           stars   View of object table to select for stars
        galaxies   View of object table to select for galaxies
          xmatch   Crossmatch of object against GAIA DR1 and WISE



In [18]:
# List columns in smash_dr1.object table
dl.schema('smash_dr1.object')


Schema: smash_dr1
 Table: object

     Column Name   Description
     -----------   -----------
            htm9   HTM index (order 9 => ~10 arcmin size)
          pix256   HEALPIX index (Nsides 256 => ~14 arcmin size)
              id   Unique ID for this object, the field name plus a running number
         fieldid   SMASH Field ID
              ra   Right Ascension (J2000.0) of source, in degrees
             dec   Declination (J2000.0) of source, in degrees
           raerr   Uncertainty in mean RA of object (arcseconds) from indiv values
          decerr   Uncertainty in mean DEC of object (arcseconds) from indiv 
                   alues
            ndet   Number of detections of this source (in ALLSRC)
       depthflag   Flag of exposure detected in: 1-shallow, 2-deep, 3-both
            umag   Weighted-average, calibrated u-band magnitude, 99.99 if no
                   detection
            uerr   Uncertainty in calibrated u-band magnitude
        uscatter   RMS scatter in u 

<a class="anchor" id="dbcommands.list"></a>
### List MyDB Tables
Use the `dl.listdb()` command to list all of the tables in your MyDB.  You can list all tables or identify individual ones.

In [19]:
# List the MyDB tables
dl.listdb()

'test test2'

<a class="anchor" id="dbcommands.droptable"></a>
### Drop a MyDB Table
Use the `dl.droptable()` command to drop/delete a single MyDB table.

In [20]:
# Drop a table in MyDB
dl.droptable('test')

Table 'test' was dropped.


<a class="anchor" id="dbcommands.exporttable"></a>
### Export a MyDB Table to a File
Use the `dl.exporttable()` command to export or copy a MyDb table to a file in your VOSpace.  The available output formats are fits, csv and hdf5.

In [21]:
# Export a table to a file
dl.exporttable('test2','vos://myresult.fits',fmt='fits')

Table 'test2' not found.


<a class="anchor" id="dbcommands.siaquery"></a>
### Simple Image Access (SIA) Search
Use the `dl.siaquery()` command to perform a Simple Image Access (SIA) search.  This will return a list of all the available images in the NOAO archive that cover a given point in the sky with a search radius.

In [22]:
# Get all images that cover ra=100.0, dec=+20.0 within a radius of 0.1 deg
list = dl.siaquery(5.0, 10.0, 0.2)

The image list contains 19 entries


<a class="anchor" id="storage"></a>
# Handling files in your virtual storage space
Next, let's learn how to deal with files in your Data Lab virtual storage (VOSpace).  This is a designated space on the DL server for your files that you can use and manipulate as you please.  There are many unix-like commands to work on these files: `ls, cp, mv, rm, mkdir, rmdir, ln`.  There are also commands to sync files with your local file system (`put` and `get`) and copying/saving a file from a URL into VOSpace (`copyurl`).  Finally there are commands for loading files into python memory (`load`) and saving data in python to files (`save`).

<a class="anchor" id="storage.ls"></a>
### File Listing
Let's start with a basic directory listing with `dl.ls()`.  This lists the file names and directory names at the main level.  Adding `verbose=True` gives more information.

In [23]:
# basic listing
dl.ls()

public/  results.hdf5  test.csv  test.fits  test.txt  tmp/ 


In [24]:
# more verbose listing
dl.ls(verbose=True)

drw-rw----  dnidever    512B  02 Feb 2017 00:00:00  public/  
-rw-rw----  dnidever  152.9K  02 Jun 2017 13:03:07  results.hdf5  
-rw-rw----  dnidever  234.4K  01 Jun 2017 12:20:52  test.csv  
-rw-rw----  dnidever  317.8K  01 Jun 2017 12:21:13  test.fits  
-rw-rw----  dnidever  234.4K  01 Jun 2017 12:20:52  test.txt  
drw-rw----  dnidever    512B  02 Feb 2017 00:00:00  tmp/  


<a class="anchor" id="storage.unixcmds"></a>
### Unix-like File Commands - cp, mv, rm, mkdir, rmdir, ln
The most important unix file manipulation commands are available in Data Lab VOSpace. `cp` - copy a file; `mv` - move or rename a file; `rm` - delete a file; `mkdir` - make a new directory; `rmdir` - remove a directory; `ln` - make a symbolic link.

In [25]:
# Copy a file
dl.cp('test.fits','test2.fits')
dl.ls()

public/  results.hdf5  test.csv  test.fits  test.txt  test2.fits  tmp/ 


In [26]:
# Move a file
dl.mv('test2.fits','newtest2.fits')
dl.ls()

newtest2.fits  public/  results.hdf5  test.csv  test.fits  test.txt  tmp/ 


In [27]:
# Delete a file
dl.rm('newtest2.fits')
dl.ls()

public/  results.hdf5  test.csv  test.fits  test.txt  tmp/ 


In [28]:
# Make a new directory
dl.mkdir('newdirectory')
dl.ls()

newdirectory/  public/  results.hdf5  test.csv  test.fits  test.txt  tmp/ 


In [29]:
# Delete a directory
dl.rmdir('newdirectory')
dl.ls()

public/  results.hdf5  test.csv  test.fits  test.txt  tmp/ 


In [30]:
# Create a symbolic link
dl.ln('myresult.fits','link_to_myresult')
dl.ls(verbose=True)

lrw-rw----  dnidever      0B  02 Jun 2017 15:15:54  link_to_myresult -> /myresult.fits  
drw-rw----  dnidever    512B  02 Feb 2017 00:00:00  public/  
-rw-rw----  dnidever  152.9K  02 Jun 2017 13:03:07  results.hdf5  
-rw-rw----  dnidever  234.4K  01 Jun 2017 12:20:52  test.csv  
-rw-rw----  dnidever  317.8K  01 Jun 2017 12:21:13  test.fits  
-rw-rw----  dnidever  234.4K  01 Jun 2017 12:20:52  test.txt  
drw-rw----  dnidever    512B  02 Feb 2017 00:00:00  tmp/  


<a class="anchor" id="storage.sync"></a>
### Copying local files to/from VOSpace
You can easily copy files from your local file system to VOSpace or vice versa with the `put` and `get` commands.  Let's put a file from our local file system called `file.fits` to our VOSpace.

In [31]:
# Copy a file to VOSpace
dl.put('test1.fits','vos://')

(1 / 1) test1.fits -> vos://test1.fits


In [32]:
# Check that the file is there
dl.ls()

public/  results.hdf5  smash_test.csv  test.csv  test.fits  test.txt  test1.fits  tmp/ 


Now let's get a file called `results.fits` from the VOSpace using `get`.

In [33]:
# Get a file from VOSpace
dl.get('vos://test1.fits','localtest1.fits')
!ls localtest1.fits

(1/1) [====================] [ 163.1K] test1.fits
localtest1.fits


<a class="anchor" id="storage.copyurl"></a>
### Copying File from URL
You can also easily copy a file into VOSpace using a URL using `copyurl`.

In [34]:
# Download a webpage
dl.copyurl('http://datalab.noao.edu/index.html','vos://webpage.html')
dl.ls()

public/  results.hdf5  smash_test.csv  test.csv  test.fits  test.txt  test1.fits  tmp/  webpage.html 


<a class="anchor" id="storage.load"></a>
### Loading files into python
There is an easy command to load files (either local or in VOSpace) into python memory using the `load` command.  The supported input files are: fits, hdf5, csv and ascii.  There are many different output formats: csv, ascii (tab-delimited), string, numpy array, numpy structarray, pandas, astropy table, and astropy votable.

In [35]:
# Load file vos://output.fits into a pandas data frame
df = dl.load('vos://test10.fits',fmt='pandas')
type(df)

pandas.core.frame.DataFrame

<a class="anchor" id="storage.save"></a>
### Saving data from python to files
Saving data from python to a file is just as easy using `save`.  The supported python data types are: csv, ascii (tab-delimited), string, numpy array, numpy structarray, pandas, astropy table, and astropy votable.  The supported output format depends somewhat on the data type but generally are: csv, fits, and hdf5.  The output format does not need to be specified if it can be determined from the filename extension.

In [36]:
# Save a pandas data frame 'df' to file vos://results.hdf5 
dl.save(df, 'vos://results2.hdf5')
dl.ls()

public/  results.hdf5  results2.hdf5  smash_test.csv  test.csv  test.fits  test.txt  test1.fits  tmp/  webpage.html 
